# LSTM Stock Predictor Using Closing Prices

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin closing prices to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [13]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [14]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [15]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('Resources/btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [16]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('Resources/btc_historic.csv', index_col="date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [17]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
date,,
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [18]:
df.head()

,fng_value,Close
date,,
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


In [19]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window -1  ):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [20]:
# Predict Closing Prices using a 10 day window of previous closing prices
# Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
window_size = 10

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)


In [21]:
# Use 70% of the data for training and the remaineder for testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size = 0.7)



In [23]:
from sklearn.preprocessing import MinMaxScaler
# Use the MinMaxScaler to scale data between 0 and 1.
# Creating the scaler instance
X_scaler = MinMaxScaler() 
y_scaler = MinMaxScaler()

# fitting the minmax scaler to the features training set 
X_scaler.fit(X)

# transforming the features traning and test sets using the scaler instance
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

# fitting the minmax scaler to the taraget training set 
y_scaler.fit(y)

# transforming the target traning and test sets using the scaler instance
y_train = y_scaler.transform(y_train)
y_test = y_scaler.transform(y_test)

In [ ]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1] ,1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],1))
print (f"X_train sample values:\n{X_train[:5]} \n")
print (f"X_test sample values:\n{X_test[:5]}")

---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [40]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# Note: The dropouts help prevent overfitting
# Note: The input shape is the number of time steps and the number of indicators
# Note: Batching inputs has a different input shape of Samples/TimeSteps/Features

# Assign the model to a variable model 
model = Sequential()

# creating the dropout fraction and number of units for later trial and error testing
number_units = 45
dropout_function = 0.2

# add the first LSTM layer
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1)))

model.add(Dropout(dropout_function))

#add layer 2 
model.add(LSTM(
    units = number_units,
    
))
model.add(Dropout(dropout_function))

# output layer 
model.add(Dense(1))
   

In [41]:
# Compile the model
model.compile(optimizer="adam", loss = "mean_squared_error") 

In [42]:
# Summarize the model
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 10, 45)            8460      
                                                                 
 dropout_4 (Dropout)         (None, 10, 45)            0         
                                                                 
 lstm_5 (LSTM)               (None, 45)                16380     
                                                                 
 dropout_5 (Dropout)         (None, 45)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 46        
                                                                 
Total params: 24,886
Trainable params: 24,886
Non-trainable params: 0
_________________________________________________________________


In [43]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended

model.fit(X_train, y_train, epochs=20, shuffle=False, batch_size=3, verbose=1)

Epoch 1/20
124/124 [==============================] - 7s 10ms/step - loss: 0.0137
Epoch 2/20
124/124 [==============================] - 1s 9ms/step - loss: 0.0073
Epoch 3/20
124/124 [==============================] - 1s 11ms/step - loss: 0.0063
Epoch 4/20
124/124 [==============================] - 1s 11ms/step - loss: 0.0060
Epoch 5/20
124/124 [==============================] - 1s 11ms/step - loss: 0.0058
Epoch 6/20
124/124 [==============================] - 1s 10ms/step - loss: 0.0058
Epoch 7/20
124/124 [==============================] - 1s 10ms/step - loss: 0.0047
Epoch 8/20
124/124 [==============================] - 1s 10ms/step - loss: 0.0046
Epoch 9/20
124/124 [==============================] - 1s 10ms/step - loss: 0.0045
Epoch 10/20
124/124 [==============================] - 1s 10ms/step - loss: 0.0041
Epoch 11/20
124/124 [==============================] - 1s 10ms/step - loss: 0.0037
Epoch 12/20
124/124 [==============================] - 1s 10ms/step - loss: 0.0040
Epoch 13/20
12

---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [44]:
X_test[0]

array([0.1736277 , 0.18745615, 0.1884912 , 0.20335677, 0.21295618,
       0.20337119, 0.21548803, 0.18759355, 0.19099616, 0.19090941])

In [45]:
# Evaluate the model
model.evaluate(X_test, y_test)

5/5 [==============================] - 1s 5ms/step - loss: 0.0016


0.0016366930212825537

In [46]:
X_test

array([[0.1736277 , 0.18745615, 0.1884912 , ..., 0.18759355, 0.19099616,
        0.19090941],
       [0.32841083, 0.30655104, 0.31160328, ..., 0.3360786 , 0.33980458,
        0.3377758 ],
       [0.786895  , 0.63950804, 0.66767832, ..., 0.73272683, 0.68399933,
        0.67552782],
       ...,
       [0.3452215 , 0.33635859, 0.33435044, ..., 0.33507044, 0.33367799,
        0.33310365],
       [0.81240436, 0.74216413, 0.82119191, ..., 0.66866791, 0.65879059,
        0.7327919 ],
       [0.35910882, 0.36330271, 0.36031121, ..., 0.39195744, 0.41018946,
        0.42029194]])

In [47]:
# Make some predictions
predicted = model.predict(X_test)


In [48]:
# Recover the original prices instead of the scaled version
predicted_prices =  y_scaler.inverse_transform(predicted)
real_prices =   y_scaler.inverse_transform(y_test)

In [52]:
# Create a DataFrame of Real and Predicted values and difference between them
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel(),
    "Difference": real_prices.ravel() - predicted_prices.ravel()
}, index = df.index[-len(real_prices): ])

stocks.head()

,Real,Predicted,Difference
date,,,
2019-02-20,5165.589844,5241.818359,-76.228515
2019-02-21,6261.479980,6592.841309,-331.361329
2019-02-22,9882.429688,9892.421875,-9.992187
2019-02-23,7371.959961,8073.342285,-701.382324
2019-02-24,6762.060059,6501.997559,260.062500


In [71]:
# Plot the real vs predicted values as a line chart / Add the diff plot to visualize the error
stocks.hvplot(x ='date', y=['Real', 'Predicted']) + stocks.hvplot.bar( y='Difference')

:Layout
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Bars.I      :Bars   [date]   (Difference)